In [34]:
from PIL import Image
import h5py
import anubisPlotUtils as anPlot
import json
import numpy as np
import os
import hist as hi
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.colors as colors
matplotlib.use('TkAgg')  # or 'Qt5Agg', 'GTK3Agg', etc.
import mplhep as hep
hep.style.use([hep.style.ATLAS])
import sys
import AnalysisToolAnubis as AT
import pandas as pd
import tkinter as tk
from tkinter import ttk
from pandasgui import show
from matplotlib.patches import Circle
from matplotlib.backends.backend_pdf import PdfPages
from scipy.signal import convolve2d
from scipy.ndimage import label, find_objects
from scipy.spatial.distance import pdist, squareform
from matplotlib.backends.backend_pdf import PdfPages
import ipywidgets as widgets
from IPython.display import display


In [4]:
#Importing the data
thisData = AT.importDatafile('C:\\Users\\Peter\\OneDrive - University of Cambridge\\Desktop\\Project Excel Work\\PartIIIRPC\\ProAnubis_CERN\\ProAnubisData\\60sRun_24_3_4.h5')


In [5]:
mapping = {
    0: {
        (0, 31): ('rpc3', 'eta'),
        (32, 63): ('rpc3', 'phi1'),
        (64, 95): ('rpc3', 'phi2'),
        (96, 127): ('rpc2', 'eta'),
    },
    1: {
        (0, 31): ('rpc2', 'phi1'),
        (32, 63): ('rpc2', 'phi2'),
        (64, 95): ('rpc1', 'eta'),
        (96, 127): ('rpc1', 'phi1'),
    },
    2: {
        (0, 31): ('rpc1', 'phi2'),
        (32, 63): ('rpc4', 'eta'),
        (64, 95): ('rpc4', 'phi1'),
        (96, 127): ('rpc4', 'phi2'),
    },
    3: {
        (0, 31): ('rpc6', 'eta'),
        (32, 63): ('rpc6', 'phi1'),
        (64, 95): ('rpc6', 'phi2'),
        (96, 127): ('rpc5', 'eta'),
    },
    4: {
        (0, 31): ('rpc5', 'phi1'),
        (32, 63): ('rpc5', 'phi2'),
    },
}

In [6]:
def assign_to_coordinates(tdc, channel, mapping):
    for channel_range, (rpc, side) in mapping.get(tdc, {}).items():
        if channel_range[0] <= channel <= channel_range[1]:
            offset = channel - channel_range[0]
            if side == 'eta':
                return [rpc, 'eta', [0, offset]]
            elif side in ['phi1', 'phi2']:
                return [rpc, 'phi', [offset, 0]]
    return ['Null', 'Null', [tdc, channel]]  # If no match found

In [7]:
def remake_data(Data, mapping):
    full_data = []
    for tdc in range(0, len(Data)):
        for events in range(0, len(Data[tdc])): 
            for hits in Data[tdc][events]:
                channel = (hits>>24)&0x7f
                coord = assign_to_coordinates(tdc, channel, mapping)
                event_time = hits&0xfffff
                
            
                
                full_data.append({
                    'tdc': tdc,
                    'event number': events,
                    'channel':channel,
                    'rpc number': coord[0],
                    'strip direction': coord[1],
                    'hit coordinate': coord[2],
                    'time': event_time
                })
    df = pd.DataFrame(full_data)
        
    return df
    

In [8]:
df = remake_data(thisData, mapping)


In [22]:
show(df)


PandasGUI INFO — pandasgui.gui — Opening PandasGUI
INFO:pandasgui.gui:Opening PandasGUI


The instruction was unclear, what does it meant by three coincidence required in the email? possibly three different RPC hit required for the trigger to record... The reason of the event time being mostly around 200 arbitrary value, is probably the time takes for the computer to react and record the correcponding time, I won't trust that value to be the exact time, but it will be interesting to see if there are correlation between those time and rpc number

In [9]:
event_rpc_dict = {}

for event in df['event number'].unique():
    rpcs_involved = df.loc[df['event number'] == event, 'rpc number'].unique()
    event_rpc_dict[event] = rpcs_involved

events_with_less_than_3_rpcs = 0

for event, rpcs in event_rpc_dict.items():
    if len(rpcs) < 3:
        events_with_less_than_3_rpcs += 1

print(events_with_less_than_3_rpcs)

8


In [10]:
event_counts = df['event number'].value_counts()

events_less_than_3_occurrences = event_counts[event_counts < 3]

num_events_less_than_3 = len(events_less_than_3_occurrences)

print(num_events_less_than_3)

0


Hence it might not have been between rpcs, try if it is between number of simulateous events
Well, that is self explainatary, and since we have 19473 entries, each event contains roughly 30.19 entries, didn't expect to be this tbh

Well it doesn't stop data analysis
total 576 strips, 19473 interaction recorded, if all through, this is about 0.56 Hz of dark counts, which is slightly lower than the expected 1Hz.

Checking if there are any broken channels (or strips)

In [11]:
all_combinations = []
for tdc in range(5):  # tdc0 to tdc4
    tdcnumber = tdc
    max_channel = 63 if tdc == 4 else 127
    for channel in range(max_channel + 1):
        all_combinations.append((tdc, channel))

all_combinations_df = pd.DataFrame(all_combinations, columns=['tdc', 'channel'])
missing_combinations = pd.merge(all_combinations_df, df, on=['tdc', 'channel'], how='left', indicator=True)
missing_combinations = missing_combinations[missing_combinations['_merge'] == 'left_only']
missing_combinations = missing_combinations[['tdc', 'channel']]
show(missing_combinations)

PandasGUI INFO — pandasgui.gui — Opening PandasGUI
INFO:pandasgui.gui:Opening PandasGUI


tdc0 channel 31
tdc0 channel 107
tdc0 channel 111
tdc2 channel 41
tdc3 channel 51
These never fired, for a 30 second run and if the dark count is stochastic with poisson distribution, it is very unlikely this will happen, hence they are probably dead...

In [12]:
#slightly different way to store data, but does the same thing, it goes like rpc - event - strip
etaHits, phiHits = AT.divideHitCountsByRPC(thisData)

In [13]:
def countCoincidencesEvents(primRpc,SecRpc, winSize=5, minHit=3, maxHit=10):
    coincArray = [0 for channel in range(len(primRpc[0]))]
    for idx, event in enumerate(primRpc):
        for channel in range(len(event)):
            nHits = 0
            for itr in range(channel-int(winSize/2.), channel+int(winSize/2.)+1):
                if itr>=0 and itr<len(event):
                    if event[channel]>0:
                        nHits = nHits+1
            if nHits>=minHit and sum(event)<=maxHit:
                if sum(SecRpc[idx])>=minHit and sum(SecRpc[idx])<=maxHit:
                    coincArray[channel]=coincArray[channel]+1
    return coincArray

In [14]:
coincidence = {}
#The countCoincidences defined a window sieze, in which it says if the number of events within this window size is more than 3 and less than 10 hits occured per event (a cluster), 
# and if the second strip also had a a total event more than 3 and less than 10, 
# then it is recorded as a successful coincidence
for i in range(5):
    result_key = f'etaHits{i}_phiHits{i}' 
    coincidence[result_key] = AT.countCoincidences(etaHits[i], phiHits[i])
    coincidence[result_key] = AT.countCoincidences(phiHits[i], etaHits[i])
#This is useful as it gives an idea how many successful hits were observed at whatever places
print(coincidence)

{'etaHits0_phiHits0': [23, 1, 2, 2, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 4, 5, 8, 6, 2, 7, 7, 4, 8, 11, 7, 4, 4, 6, 8, 6, 3, 4, 6, 7, 9, 15, 15, 11, 9, 6, 6, 2, 2, 0, 6, 2, 7, 6, 6, 7, 4, 3, 7, 4, 4, 5, 5, 3, 5, 5, 6, 6], 'etaHits1_phiHits1': [4, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 2, 5, 4, 1, 6, 6, 4, 5, 5, 6, 9, 7, 7, 9, 5, 5, 2, 2, 1, 0, 2, 1, 2, 3, 3, 7, 7, 2, 3, 3, 3, 2, 1, 4, 7, 3, 11, 10, 9, 5, 4, 6, 10, 4, 9, 7, 8, 8, 7, 12, 11, 7], 'etaHits2_phiHits2': [24, 5, 5, 4, 4, 2, 5, 4, 6, 7, 8, 9, 6, 6, 5, 7, 10, 9, 6, 6, 3, 5, 5, 5, 5, 5, 6, 4, 3, 3, 6, 4, 8, 5, 7, 6, 7, 9, 5, 13, 8, 9, 10, 8, 2, 2, 6, 3, 12, 8, 9, 7, 4, 3, 6, 5, 11, 6, 9, 6, 6, 6, 8, 5], 'etaHits3_phiHits3': [0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 2, 2, 1, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 1, 1, 3, 2, 1, 0, 1, 1, 2, 2, 2, 2, 0, 0, 2, 2, 1, 1, 1, 1, 1, 2, 2, 2, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0], 'etaHits4_phiHits4': [9, 3, 4, 3, 4, 3, 2, 2, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 3, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 2, 3,

Looking first at 1D data for the strippes, without any time associated with it (treating events as independant discreet series)
we can quantify a clustering metric

In [15]:
def calculate_cluster_metrics(event, minHit=3, maxHit=10):
    cluster_sizes = []
    cluster_locations = []
    current_cluster_size = 0
    start_index = None  
    
    for index, i in enumerate(event):
        if i != 0:
            current_cluster_size += i
            if start_index is None:
                start_index = index  
        elif i == 0 and current_cluster_size >= minHit and current_cluster_size < maxHit:
            cluster_midpoint = (start_index + index + 1) / 2.0  
            cluster_locations.append(cluster_midpoint)
            cluster_sizes.append(current_cluster_size)
            current_cluster_size = 0
            start_index = None 
        elif i == 0:
            current_cluster_size = 0
            start_index = None 
            
    # Handle the last cluster if it meets the criteria
    if current_cluster_size >= minHit and current_cluster_size < maxHit:
        cluster_midpoint = (start_index + len(event) + 1) / 2.0 
        cluster_locations.append(cluster_midpoint)
        cluster_sizes.append(current_cluster_size)
    
    return cluster_locations, cluster_sizes


In [16]:
columns = ['RPC Number', 'Event Number', 'cluster_locations', 'Cluster_size']
dfcluseta = pd.DataFrame(columns=columns)
dfclusphi = pd.DataFrame(columns=columns)

for i, rpc_data in enumerate(etaHits):
    for j, event in enumerate(rpc_data):
        cluster_locations, cluster_sizes = calculate_cluster_metrics(event)
        for _ in range(len(cluster_sizes)):
            
            dfcluseta = dfcluseta.append({'RPC Number': int(i), 'Event Number': int(j), 'cluster_locations': cluster_locations[_], 'Cluster_size': int(cluster_sizes[_])}, ignore_index=True)

for i, rpc_data in enumerate(phiHits):
    for j, event in enumerate(rpc_data):
        cluster_locations, cluster_sizes = calculate_cluster_metrics(event)
        for _ in range(len(cluster_sizes)):
            
            dfclusphi = dfclusphi.append({'RPC Number': int(i), 'Event Number': int(j), 'cluster_locations': cluster_locations[_], 'Cluster_size': int(cluster_sizes[_])}, ignore_index=True)

C:\Users\Peter\AppData\Local\Temp\ipykernel_1808\1678047686.py:10: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\Peter\AppData\Local\Temp\ipykernel_1808\1678047686.py:10: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\Peter\AppData\Local\Temp\ipykernel_1808\1678047686.py:10: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\Peter\AppData\Local\Temp\ipykernel_1808\1678047686.py:10: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\Peter\AppData\Local\Temp\ipykernel_1808\1678047686.py:10: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat 

In [126]:
show(dfclusphi)

PandasGUI INFO — pandasgui.gui — Opening PandasGUI
INFO:pandasgui.gui:Opening PandasGUI


In [18]:
rpc_dfseta = {rpc: dfcluseta[dfcluseta['RPC Number'] == rpc].reset_index(drop=True) for rpc in dfcluseta['RPC Number'].unique()}
rpc_dfsphi = {rpc: dfclusphi[dfclusphi['RPC Number'] == rpc].reset_index(drop=True) for rpc in dfclusphi['RPC Number'].unique()}

def plot_rpc_histograms(rpc_dfseta, rpc_dfsphi):
    with PdfPages('C:\\Users\\Peter\\OneDrive - University of Cambridge\\Desktop\\Project Excel Work\\PartIIIRPC\\ProAnubis_CERN\\ProAnubis Plots\\rpc_cluster_analysis.pdf') as pdf:
        for rpc in rpc_dfseta.keys():
            df_eta = rpc_dfseta[rpc]
            df_phi = rpc_dfsphi[rpc]
            for df, dataset_name in zip([df_eta, df_phi], ['eta', 'phi']):
                all_cluster_locations = []
                all_cluster_sizes = []
                
                for _, row in df.iterrows():
                    locs = row['cluster_locations']
                    sizes = row['Cluster_size']
                    
                    if not isinstance(locs, list):
                        locs = [locs] if not pd.isnull(locs) else []
                    if not isinstance(sizes, list):
                        sizes = [sizes] if not pd.isnull(sizes) else []
                    
                    all_cluster_locations.extend(locs)
                    all_cluster_sizes.extend(sizes)
                
                locations_series = pd.Series(all_cluster_locations)
                sizes_series = pd.Series(all_cluster_sizes)
                
                location_frequencies = locations_series.value_counts().sort_index()
                size_frequencies = sizes_series.value_counts().sort_index()
                
                fig, axs = plt.subplots(1, 2, figsize=(12, 6))
                fig.suptitle(f'RPC {rpc} - {dataset_name.upper()}')

                if not location_frequencies.empty:
                    axs[0].bar(location_frequencies.index, location_frequencies.values, color='skyblue', edgecolor='black')
                axs[0].set_title(f'Cluster Locations')
                axs[0].set_xlabel('Cluster Location')
                axs[0].set_ylabel('Frequency')

                if not size_frequencies.empty:
                    axs[1].bar(size_frequencies.index, size_frequencies.values, color='salmon', edgecolor='black')
                axs[1].set_title(f'Cluster Sizes')
                axs[1].set_xlabel('Cluster Size')
                axs[1].set_ylabel('Frequency')
                
                if dataset_name == 'phi':
                    axs[0].set_xlim(0, 64)
                    # axs[1].set_xlim(0, 64)
                if dataset_name == 'eta':
                    axs[0].set_xlim(0, 32)
                    # axs[1].set_xlim(0, 32)
                    
                plt.tight_layout(rect=[0, 0.03, 1, 0.95])
                pdf.savefig(fig)
                plt.close()


In [17]:
confirmation = input("This action might cause a large amount of folders to be created at unwanted areas in your computer, please make sure you have changed the pdf directory before proceeding. Are you ready to proceed? (yes/no):")


if confirmation.lower() == 'yes':
    print("Proceeding with the operation...")
    plot_rpc_histograms(rpc_dfseta, rpc_dfsphi)

else:
    print("Operation canceled.")

Proceeding with the operation...


Cool, now we can extend this into 2D, and try to find circles around interesting points, again, concerned about locations and size. Maybe we should define a kernel?
Being a good human being, I will again consider one event without thinking event time
and assume everything 100 efficient, if 2 strips cross, that is the coordinate of the hit

In [21]:
def create_hit_matrices(phiHits, etaHits):
    """
    Create hit matrices for each event in each RPC.
    
    Parameters:
    - phiHits, etaHits: Lists containing hit data for phi and eta directions.
    
    Returns:    
    - A list of matrices representing hits for each RPC layer and event.
    """
    num_rpcs, num_events = len(phiHits), len(phiHits[0])
    hit_matrices = [[None for _ in range(num_events)] for _ in range(num_rpcs)]
    
    for z in range(num_rpcs):
        for l in range(num_events):
            phi_activations = phiHits[z][l]
            eta_activations = etaHits[z][l]
            
            hit_matrix = np.zeros((64, 32), dtype=int)
            for y in range(64):
                for x in range(32):
                    if phi_activations[y] == 1 and eta_activations[x] == 1:
                        hit_matrix[y, x] = 1
                        
            hit_matrices[z][l] = hit_matrix
            
    return hit_matrices

In [33]:
def refine_clusters(hit_matrix, kernel_size=(2,2), leniency=1, minHits=3, maxHits=10):
    """
    Refine cluster identification to calculate the midpoint, rough size (area), and estimate the radius of each cluster.
    
    Parameters:
    - hit_matrix: 2D numpy array representing hits in the RPC detector.
    - kernel_size: Tuple indicating the dimensions of the kernel (height, width).
    - leniency: Percentage of the kernel that needs to be filled to consider it a cluster (0-1).
    - minHits, maxHits: Minimum and maximum number of hits to consider a set of hits a cluster.
    
    Returns:
    - A list of tuples where each tuple represents a cluster's properties: (midpoint row, midpoint column, cluster size, radius).
    """
    kernel = np.ones(kernel_size)
    conv_result = convolve2d(hit_matrix, kernel, mode='same', boundary='fill', fillvalue=0)
    
    threshold = kernel_size[0] * kernel_size[1] * leniency
    cluster_map = conv_result >= threshold
    
    labeled_array, num_features = label(cluster_map)
    
    clusters = []
    for i, slice_tuple in enumerate(find_objects(labeled_array)):
        cluster_indices = np.where(labeled_array == i + 1)
        cluster_slice = hit_matrix[slice_tuple]
        cluster_hits = cluster_slice.sum()

        if cluster_hits < minHits or cluster_hits > maxHits:
            continue

        # area = (slice_tuple[1].stop - slice_tuple[1].start) * (slice_tuple[0].stop - slice_tuple[0].start)
        area = cluster_hits
        radius = np.sqrt(area / np.pi)
        midpoint_y = cluster_indices[0].mean()
        midpoint_x = cluster_indices[1].mean()

        clusters.append((midpoint_y, midpoint_x, cluster_hits, radius))
    
    return clusters


Clarification on the functinos
Convolve2d:
2d convolution between 2 2d arrays, it applies a uniform kernel over the hitmatrix
and the kernel sums up the values of hits in each neighborhood definedby its size, 
effectively measuring the local density

Label:
Label identifies and labels connected components within a binary array.
Connected components are set of pixels or array elements that are connected to each other
by pixel adjacency
it analyzes the cluster_map, which are a series True and falses at locations, it labels each contiguous area of activated points by the kernel with unique identifiers

find_objects:
locates and returns a list of slices for each labeled object in an array. Each slice in the list corresponds to a minimal bounding box that contains a labeled object
label was used to connect components, find object then used to find the bounding boxes of these labeled clusters, 
it will be useful in the future, just not now

Parameters
hit_matrix: A 2D numpy array where each element represents whether there was a hit (e.g., a particle detection) at that location within the RPC detector. The value of each element can indicate the presence or absence of a hit.
kernel_size: A tuple specifying the height and width of a kernel, which is essentially a small matrix used to examine the hit matrix. The kernel scans over the hit matrix to aggregate hits within its footprint, aiding in cluster detection.
leniency: A value between 0 and 1 that determines the fraction of the kernel that needs to be filled with hits for the area to be considered part of a cluster. A higher leniency requires a denser concentration of hits to identify a cluster.
minHits and maxHits: These parameters define the minimum and maximum number of hits that a group must have to be considered a valid cluster. This range helps filter out noise (random hits) and overly large clusters that might not be useful for analysis.
Process,
similarly to what they used
Convolution with Kernel: The function first performs a 2D convolution of the hit matrix with a kernel of ones (kernel_size). This operation aggregates the number of hits within the area covered by the kernel as it scans across the entire hit matrix. The result (conv_result) is a new matrix where each element represents the aggregated hit count for the corresponding area in the original hit matrix.

Thresholding: By applying the leniency factor to the size of the kernel, a threshold is calculated. This threshold is then used to create a binary map (cluster_map), where elements are marked as 1 (true) if the aggregated hit count in the convolved result meets or exceeds the threshold, indicating potential cluster areas, or 0 (false) otherwise.

Labeling Clusters: The binary map is processed to label connected components (areas where adjacent elements are marked as 1) as individual clusters. This step assigns a unique label to each cluster, allowing them to be identified and analyzed separately.

Analyzing Clusters: For each labeled cluster, the function:

Calculates the midpoint (average position) of the hits within the cluster, providing a central point that represents the cluster's location.
Determines the cluster size as the sum of hits within the cluster, reflecting the cluster's intensity or activity level.
Estimates the cluster's radius based on its area. The area is computed from the dimensions of the bounding box surrounding the cluster, and the radius is calculated assuming the cluster area approximates a circle. This radius gives a sense of the cluster's spatial extent.
Filtering Clusters: The function filters out clusters that do not meet the specified criteria for the number of hits (minHits and maxHits). This step ensures that only clusters with a significant and reasonable amount of activity are considered for further analysis.

Output
The function returns a list of tuples, each representing a detected cluster's properties: its midpoint coordinates (y, x), the total number of hits (cluster size), and its estimated radius. This output provides a concise summary of the clusters identified in the RPC detector data, useful for subsequent analysis or visualization tasks.

I hated the fact that we need to caculate radius with respect to area, a better approach could just to do euclidean where we are counting the 2 furthest points within a cluster


In [35]:
def refine_clustersV2(hit_matrix, kernel_size=(2,2), leniency=1, minHits=3, maxHits=10):
    kernel = np.ones(kernel_size)
    conv_result = convolve2d(hit_matrix, kernel, mode='same', boundary='fill', fillvalue=0)
    
    threshold = kernel_size[0] * kernel_size[1] * leniency
    cluster_map = conv_result >= threshold
    
    labeled_array, num_features = label(cluster_map)
    
    clusters = []
    for i in range(1, num_features + 1):
        cluster_indices = np.where(labeled_array == i)
        if len(cluster_indices[0]) == 0:
            continue # Skip if no hits in cluster
        
        cluster_hits = hit_matrix[cluster_indices].sum()
        if cluster_hits < minHits or cluster_hits > maxHits:
            continue

        # Calculate the centroid
        midpoint_y = cluster_indices[0].mean()
        midpoint_x = cluster_indices[1].mean()

        # Find the diameter of the cluster (maximum distance between any two points)
        if len(cluster_indices[0]) > 1:
            points = np.column_stack(cluster_indices)
            distances = pdist(points, 'euclidean')
            diameter = np.max(distances)
            radius = diameter / 2
        else:
            radius = 0 # Single hit, radius is effectively 0
        
        clusters.append((midpoint_y, midpoint_x, cluster_hits, radius))
    
    return clusters

In [36]:
def aggregate_clusters_for_rpc(rpc, kernel_size, leniency, maxhit = 10):
    """
    Aggregate clusters across all events in an RPC to visualize the hit locations and radii.
    
    Parameters:
    - rpc: A list of hit matrices for each event in an RPC.
    - kernel_size, leniency: Parameters for refining clusters.
    
    Returns:
    - A matrix representing the aggregated cluster hits, locations, and radii for the RPC.
    """
    aggregated_clusters = []
    total_clusters = 0
    
    for event in rpc:
        clusters = refine_clustersV2(event, kernel_size, leniency, maxHits = maxhit)
        if clusters != []:
            total_clusters += len(clusters)
        
        aggregated_clusters.extend(clusters)
    
    return aggregated_clusters, total_clusters

# def plot_cumulative_heatmap(plot_matrix, rpc_index):
#     plt.imshow(plot_matrix, cmap='hot', interpolation='nearest')
#     plt.colorbar(label='Cumulative Cluster Hits')
#     plt.title(f"RPC {rpc_index} Cumulative Cluster Hits Heatmap")
#     plt.xlabel("eta")
#     plt.ylabel("phi")
#     plt.tight_layout()
    
# def plot_cumulative_contour(plot_matrix, rpc_index):
#     plt.contourf(plot_matrix, cmap='hot')
#     plt.colorbar(label='Cumulative Cluster Hits')
#     plt.title(f"RPC {rpc_index} Cumulative Cluster Hits Contour")
#     plt.xlabel("eta")
#     plt.ylabel("phi")
#     plt.tight_layout()
    
def plot_clusters_with_annotations(clusters, rpc_index):
    """
    Correctly plots clusters with their midpoints marked, color-coded by intensity,
    and circles representing their radius, with axes and aspect ratio correctly adjusted.
    """
    fig, ax = plt.subplots()

    sizes = np.array([cluster[2] for cluster in clusters]) 
    radii = np.array([cluster[3] for cluster in clusters])  
    colors = sizes  

    scatter = ax.scatter([cluster[0] for cluster in clusters],  
                         [cluster[1] for cluster in clusters],  
                         s=[cluster[2]*10 for cluster in clusters],  
                         c=colors, cmap='viridis', alpha=0.6, edgecolors='w')

    for cluster in clusters:
        circle = Circle((cluster[0], cluster[1]), cluster[3], color='red', fill=False, alpha=0.5)
        ax.add_patch(circle)

    ax.set_aspect(1.0) 

    cbar = plt.colorbar(scatter, ax=ax)
    cbar.set_label('Intensity (Hit Count)')
    plt.xlabel('Phi')  
    plt.ylabel('Yta') 
    plt.title(f'RPC {rpc_index} Cluster Visualization')

    plt.grid(True)
    plt.tight_layout()

    plt.grid(True)
    plt.tight_layout()
        

In [22]:
hit_matrix = create_hit_matrices(phiHits, etaHits)
kernel_size = (2, 2)
leniency = 1

In [67]:
# confirmation = input("This action might cause a large amount of folders to be created at unwanted areas in your computer, please make sure you have changed the pdf directory before proceeding. Are you ready to proceed? (yes/no):")


# if confirmation.lower() == 'yes':
#     print("Proceeding with the operation...")
#     with PdfPages('C:\\Users\\Peter\\OneDrive - University of Cambridge\\Desktop\\Project Excel Work\\PartIIIRPC\\ProAnubis_CERN\\ProAnubis Plots\\rpc_2D_cumulative_cluster_countour.pdf') as pdf:
#         for rpc_index, rpc in enumerate(hit_matrix):
#             rpc_plot_matrix = aggregate_clusters_for_rpc(rpc, kernel_size, leniency)
#             plt.figure(figsize=(10, 8))
#             plot_cumulative_contour(rpc_plot_matrix, rpc_index)
#             pdf.savefig()
#             plt.close()
# else:
#     print("Operation canceled.")

Proceeding with the operation...


In [68]:
# confirmation = input("This action might cause a large amount of folders to be created at unwanted areas in your computer, please make sure you have changed the pdf directory before proceeding. Are you ready to proceed? (yes/no):")


# if confirmation.lower() == 'yes':
#     print("Proceeding with the operation...")
#     with PdfPages('C:\\Users\\Peter\\OneDrive - University of Cambridge\\Desktop\\Project Excel Work\\PartIIIRPC\\ProAnubis_CERN\\ProAnubis Plots\\rpc_2D_cumulative_cluster_heatmaps.pdf') as pdf:
#         for rpc_index, rpc in enumerate(hit_matrix): 
#             rpc_plot_matrix = aggregate_clusters_for_rpc(rpc, kernel_size, leniency)
#             plt.figure(figsize=(10, 8))
#             plot_cumulative_heatmap(rpc_plot_matrix, rpc_index)
#             pdf.savefig()
#             plt.close()

# else:
#     print("Operation canceled.")

Proceeding with the operation...


In [38]:
hit_matrix = create_hit_matrices(phiHits, etaHits)
K = 2
kernel_size = (K, K)
leniency = 1
maxhit = 10
confirmation = input("This action might cause a large amount of folders to be created at unwanted areas in your computer, please make sure you have changed the pdf directory before proceeding. Are you ready to proceed? (yes/no):")
maxhits_range = list(range(10, 12, 10))

if confirmation.lower() == 'yes':
    print("Proceeding with the operation...")
    for maxhits in maxhits_range:
        with PdfPages(f'C:\\Users\\Peter\\OneDrive - University of Cambridge\\Desktop\\Project Excel Work\\PartIIIRPC\\ProAnubis_CERN\\ProAnubis Plots\\rpc_2D_cluster_2K_{maxhits}NEW.pdf') as pdf:
                for rpc_index, rpc in enumerate(hit_matrix):
                    rpc_plot_matrix, total_clusters = aggregate_clusters_for_rpc(rpc, kernel_size, leniency, maxhits) 
                    print(rpc_plot_matrix)
                    plot_clusters_with_annotations(rpc_plot_matrix, rpc_index)
                    pdf.savefig()  
                    plt.close()
                

else:
    print("Operation canceled.")

Proceeding with the operation...
[(52.5, 6.0, 4, 1.5), (30.5, 16.5, 4, 0.7071067811865476), (36.0, 5.5, 6, 1.118033988749895), (42.5, 5.5, 8, 1.5811388300841898), (11.5, 15.0, 6, 2.5), (18.0, 21.5, 6, 1.118033988749895), (31.0, 13.0, 5, 2.0), (37.5, 13.0, 10, 2.0615528128088303), (53.0, 13.0, 5, 2.0), (38.5, 18.5, 8, 1.5811388300841898), (57.5, 17.5, 4, 0.7071067811865476), (62.0, 17.5, 6, 1.118033988749895), (45.0, 3.0, 3, 1.0), (9.5, 8.0, 10, 4.5), (18.0, 8.0, 3, 1.0), (37.0, 27.0, 5, 2.0), (28.0, 12.5, 10, 2.0615528128088303), (35.5, 12.5, 8, 1.5811388300841898), (61.0, 27.0, 5, 2.0), (53.0, 19.5, 6, 1.118033988749895), (61.0, 6.5, 10, 2.0615528128088303), (61.0, 10.0, 5, 2.0), (18.0, 9.0, 3, 1.0), (18.0, 18.0, 9, 1.4142135623730951), (18.0, 23.0, 9, 1.4142135623730951), (25.5, 9.0, 4, 1.5), (30.0, 18.0, 3, 1.0), (30.0, 23.0, 3, 1.0), (20.0, 25.5, 10, 2.0615528128088303), (27.0, 25.5, 10, 2.0615528128088303), (62.0, 22.0, 9, 1.4142135623730951), (12.0, 3.5, 10, 2.0615528128088303), 

In [39]:
confirmation = input("This action might cause a large amount of folders to be created at unwanted areas in your computer, please make sure you have changed the pdf directory before proceeding. Are you ready to proceed? (yes/no): ")

if confirmation.lower() == 'yes':
    print("Proceeding with the operation...")
    total_clusters_data = {}
    maxhits_range = list(range(10, 102, 2))

    for K in range(2, 6): 
        kernel_size = (K, K)
        total_clusters_data[K] = []

        for maxhit in maxhits_range: 
            total_clusters_k_maxhit = [] 

            for rpc_index, rpc in enumerate(hit_matrix):
                rpc_plot_matrix, total_clusters = aggregate_clusters_for_rpc(rpc, kernel_size, leniency, maxhit)
                total_clusters_k_maxhit.append(total_clusters)
            
            # Average total_clusters for the current combination of K and maxhit across all RPCs
            avg_total_clusters = np.mean(total_clusters_k_maxhit)
            total_clusters_data[K].append(avg_total_clusters)
        if len(total_clusters_data[K]) != len(maxhits_range):
            print(f"Mismatch in data points for K={K}: expected {len(maxhits_range)}, got {len(total_clusters_data[K])}")

    # Plotting the results
    plt.figure(figsize=(10, 6))
    for K in total_clusters_data:
        plt.plot(maxhits_range, total_clusters_data[K], label=f'K={K}')

    plt.title('Total Clusters vs Maxhit for different K')
    plt.xlabel('Maxhit')
    plt.ylabel('Total Clusters')
    plt.legend()
    plt.grid(True)
    plt.savefig('C:\\Users\\Peter\\OneDrive - University of Cambridge\\Desktop\\Project Excel Work\\PartIIIRPC\\ProAnubis_CERN\\ProAnubis Plots\\total_clusters_vs_maxhit_info_moreNEW.pdf')
    plt.close()

Proceeding with the operation...


In [40]:
confirmation = input("This action might cause a large amount of folders to be created at unwanted areas in your computer, please make sure you have changed the pdf directory before proceeding. Are you ready to proceed? (yes/no): ")

if confirmation.lower() == 'yes':
    print("Proceeding with the operation...")
    pdf_path = 'C:\\Users\\Peter\\OneDrive - University of Cambridge\\Desktop\\Project Excel Work\\PartIIIRPC\\Cluster_Count_and_average_radius_vs_maxhit_per_RPC.pdf'
    maxhits_range = list(range(10, 2050, 25))

    with PdfPages(pdf_path) as pdf:
        for rpc_index in range(6):  # For RPC0 to RPC5
            total_clusters_data = {}
            average_radius_data = {}
            for K in range(2, 6):
                kernel_size = (K, K)
                total_clusters_data[K] = []
                average_radius_data[K] = []
                for maxhit in maxhits_range:
                    rpc = hit_matrix[rpc_index]
                    _, total_clusters = aggregate_clusters_for_rpc(rpc, kernel_size, leniency, maxhit)
                    averageradius = sum(item[3] for item in _) / total_clusters
                    total_clusters_data[K].append(total_clusters)
                    average_radius_data[K].append(averageradius)

            # Plotting the results for the current RPC
            fig, ax1 = plt.subplots(figsize=(20, 12))

            color = 'tab:red'
            ax1.set_xlabel('Maxhit')
            ax1.set_ylabel('Total Clusters', color=color)
            for K in total_clusters_data:
                ax1.plot(maxhits_range, total_clusters_data[K], label=f'K={K} Cluster', color=color)
                ax1.tick_params(axis='y', labelcolor=color)

            ax2 = ax1.twinx() 
            color = 'tab:blue'
            ax2.set_ylabel('Average Radius', color=color) 
            for K in average_radius_data:
                ax2.plot(maxhits_range, average_radius_data[K], label=f'K = {K} Avg Radius', color=color, linestyle='--')
                ax2.tick_params(axis='y', labelcolor=color)

            fig.tight_layout() 
            fig.legend(loc="upper left", bbox_to_anchor=(0.1, 0.9))
            plt.title(f'Total Clusters and average radius vs Maxhit for different K (RPC{rpc_index})')
            plt.grid(True)
            pdf.savefig() 
        plt.close()

print(f"Operation completed. The plots are saved in {pdf_path}")


Proceeding with the operation...
Operation completed. The plots are saved in C:\Users\Peter\OneDrive - University of Cambridge\Desktop\Project Excel Work\PartIIIRPC\Cluster_Count_and_average_radius_vs_maxhit_per_RPC.pdf


In [23]:
print(hit_matrix)

[[array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]]), array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]]), array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]]), array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]]), array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
  

In [29]:
A, B = aggregate_clusters_for_rpc(hit_matrix[0], (2,2), 1, 100)
print(A)
z = sum_third_elements = sum(item[3] for item in A)
print(z)

[(52.5, 6.0, 4, 1.1283791670955126), (30.5, 16.5, 4, 1.1283791670955126), (36.0, 5.5, 6, 1.381976597885342), (42.5, 5.5, 8, 1.5957691216057308), (11.5, 15.0, 6, 1.381976597885342), (18.0, 21.5, 6, 1.381976597885342), (22.5, 13.0, 60, 4.370193722368317), (31.0, 13.0, 5, 1.2615662610100802), (37.5, 13.0, 10, 1.7841241161527712), (53.0, 13.0, 5, 1.2615662610100802), (38.5, 18.5, 8, 1.5957691216057308), (59.0, 23.5, 18, 2.393653682408596), (12.5, 2.0, 20, 2.5231325220201604), (28.5, 17.5, 12, 1.9544100476116797), (57.5, 17.5, 4, 1.1283791670955126), (62.0, 17.5, 6, 1.381976597885342), (10.0, 1.5, 38, 3.477898169151024), (10.0, 5.0, 19, 2.4592453796829674), (45.0, 3.0, 3, 0.9772050238058398), (59.0, 23.5, 18, 2.393653682408596), (9.5, 8.0, 10, 1.7841241161527712), (18.0, 8.0, 3, 0.9772050238058398), (12.0, 13.5, 78, 4.982787485166879), (37.0, 27.0, 5, 1.2615662610100802), (28.0, 12.5, 10, 1.7841241161527712), (35.5, 12.5, 8, 1.5957691216057308), (59.0, 12.5, 18, 2.393653682408596), (61.0, 2

This heat map is not very useful since the clarity of the map is not easy to see. There could be a much easier way to display this graph. So just trying out if plotting in 3D will actually help
And I realised there were Major Flaws in my previous code

Now it looks good, but then here is the catch, I NEED to know what is the frequency of these clusters, becuase most likely these are just dark counts right, so it is necessary to count these clusters and compare how the number change wrt parameters given

luckily for us, the parameters given are discreet, hence heurestic search will be sufficient... Althought it is very naive, but it is just for me to kind of see if what I have defined are realistic. 

In [152]:
def evaluate_events(rpc, kernel_size, leniency, minHits, maxHits, target_clusters=30):
    total_clusters = 0
    for event in rpc:
        clusters = refine_clusters(event, kernel_size, leniency, minHits, maxHits)
        if clusters != []:
            total_clusters += len(clusters)
    return abs(total_clusters - target_clusters)

def count_event(rpc, kernel_size, leniency, minHits, maxHits, target_clusters=30):
    total_clusters = 0
    for event in rpc:
        clusters = refine_clusters(event, kernel_size, leniency, minHits, maxHits)
        if clusters != []:
            total_clusters += len(clusters)
    return total_clusters

def optimize_parameters(rpc, initial_params, param_bounds, target_clusters=30, iterations=100):
    best_params = initial_params.copy()
    best_difference = evaluate_events(rpc, **best_params, target_clusters=target_clusters)
    
    def adjust_param(param, delta, bounds):
        new_param = param + delta
        return max(min(new_param, bounds[1]), bounds[0])

    for _ in range(iterations):
        for param, bounds in param_bounds.items():
            current_value = best_params[param]
            if param == "kernel_size":
                for delta in [-1, 1]: 
                    new_kernel_size = (adjust_param(current_value[0], delta, bounds[0]),
                                       adjust_param(current_value[1], delta, bounds[1]))
                    best_params[param] = new_kernel_size
                    new_difference = evaluate_events(rpc, **best_params, target_clusters=target_clusters)
                    if new_difference < best_difference:
                        best_difference = new_difference
                    else:
                        best_params[param] = current_value  
            else:
                for delta in [-0.1, 0.1] if param == "leniency" else [-1, 1]:
                    new_value = adjust_param(current_value, delta, bounds)
                    best_params[param] = new_value
                    new_difference = evaluate_events(rpc, **best_params, target_clusters=target_clusters)
                    if new_difference < best_difference:
                        best_difference = new_difference
                    else:
                        best_params[param] = current_value  
    
    return best_params, best_difference
for index, rpc in enumerate(hit_matrix):
    initial_params = {
        "kernel_size": (2, 2),
        "leniency": 0.7,
        "minHits": 3,
        "maxHits": 10
    }
    param_bounds = {
        "kernel_size": [(1, 1), (5, 5)],
        "leniency": [0.5, 0.9],
        "minHits": [2, 5],
        "maxHits": [8, 15]
    }


In [ ]:
best_params, best_difference = optimize_parameters(rpc, initial_params, param_bounds, target_clusters=30)

print("fBest Parameters Found for rpc{i}:", best_params)
print("fBest Difference for rpc{i}:", best_difference)

Just realised, we don't even have a target_clusters number, but by looking at the dark coincidence, we could certainly have anticipated some to be fake particles, this is not desireble. 

In [153]:
print(count_event(hit_matrix[0], (2,2), 1, 3, 10))
print(count_event(hit_matrix[1], kernel_size, 1, 3, 10))
print(count_event(hit_matrix[2], kernel_size, 1, 3, 10))
print(count_event(hit_matrix[3], kernel_size, 1, 3, 10))
print(count_event(hit_matrix[4], kernel_size, 1, 3, 10))
print(count_event(hit_matrix[5], kernel_size, 1, 3, 10))

134
86
123
37
25
19


This was a good graph, it showed how some strips inately have more noise than others

However when camparing with the previous analysis, discrepencies were seen. 
I am not sure what caused it, sometimes the common sense checks out with the other data, sometimes you simply see discrepencies. 
Ahh shit, I interpreted the min and max hit wrong...

Moment of realisation
1. Liniency completely useless
2. increasing kernel effectively filters out smaller dark counts, almost as if you are increasing threhold
3. Effectively the algorithm shrunk the size of the frame simply by how kernels work (effectively why you need lineancy...) But in the grand scope, a 2X2 kernel with lineincy 1/2 is equivalent of 1X2 kernel...
4. Well All things said, this will be useful for locating where the actual incidence is, however a cluster of . However everytime I see 10 strips go off, I always doubt myself, could this really be the work of dark counts...
